In [5]:
# Dependencies
import pandas as pd
import numpy as np
import time
from citipy import citipy
import requests
from config import weather_api_key

# API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [2]:
# 3. Create a new list of 2000 random coordinates

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Add the latitudes and longitudes to a list
coordinates = list(zip(lats, lngs))
# coordinates

In [3]:
# 4. Get the nearest cities with citipy
cities = []

# ID the nearest city for each lat/long combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list

    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


751

In [6]:
# 5. Perform the API call with the OpenWeatherApp

city_data = []

# Print the beginning of the logging.

print("----------------------------------")
print("     Beginning Data Retrieval     ")
print("----------------------------------")


# Create counters

record_count = 1
set_count = 1

# Loop through all of the cities in the list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(20)

    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ", "+")

    # Log the URL, record, and set numbers and the city

    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count

    record_count += 1

# 6. Retreive the info from the API call - country, lat/long, max temp, humidity, cloudiness, wind speed, description
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # Description is located in a dictionary within a list - what a pain in the ass
        city_description = [i["description"] for i in city_weather["weather"]]



        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description
                          })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------------")
print("      Data Retrieval Complete      ")
print("-----------------------------------")


----------------------------------
     Beginning Data Retrieval     
----------------------------------
Processing Record 1 of Set 1 | kavieng
Processing Record 2 of Set 1 | shellbrook
Processing Record 3 of Set 1 | roald
Processing Record 4 of Set 1 | thompson
Processing Record 5 of Set 1 | haibowan
City not found. Skipping...
Processing Record 6 of Set 1 | aasiaat
Processing Record 7 of Set 1 | avarua
Processing Record 8 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 9 of Set 1 | khani
Processing Record 10 of Set 1 | souillac
Processing Record 11 of Set 1 | puerto ayora
Processing Record 12 of Set 1 | keti bandar
Processing Record 13 of Set 1 | victoria
Processing Record 14 of Set 1 | utiroa
City not found. Skipping...
Processing Record 15 of Set 1 | pangnirtung
Processing Record 16 of Set 1 | mnogovershinnyy
Processing Record 17 of Set 1 | caravelas
Processing Record 18 of Set 1 | the valley
Processing Record 19 of Set 1 | esperance
Processing Record 20 of Set

In [7]:
# Add the array of dictionaries a new DF

city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kavieng,PG,-2.5744,150.7967,82.38,71,98,8.08,[overcast clouds]
1,Shellbrook,CA,53.2168,-106.4011,64.80,45,8,3.56,[clear sky]
2,Roald,NO,62.5833,6.1333,59.32,94,75,2.30,[broken clouds]
3,Thompson,CA,55.7435,-97.8558,64.56,36,75,8.05,[broken clouds]
4,Aasiaat,GL,68.7098,-52.8699,41.38,69,31,14.61,[scattered clouds]
5,Avarua,CK,-21.2078,-159.7750,80.65,69,9,5.75,[clear sky]
6,Khani,GE,41.9563,42.9566,60.37,96,99,1.28,[overcast clouds]
7,Souillac,MU,-20.5167,57.5167,69.22,70,80,18.70,[light rain]
8,Puerto Ayora,EC,-0.7393,-90.3518,70.56,89,26,8.01,[scattered clouds]
9,Keti Bandar,PK,24.1447,67.4497,84.78,83,100,14.81,[light rain]


In [9]:
# 8. Export the data to the Database folder
output_file = "WeatherPy_Database.csv"

city_data_df.to_csv(output_file, index_label="City_ID")
